In [10]:
import json
import numpy as np
import os
from skimage.io import imread
import matplotlib.pyplot as plt
from utils.maps_creation import calculate_sc,calculate_smooth,calculate_soft
from utils.metrics_computation import calculate_max_threshold


dir = '/home/laura/Documents/dataset/PANDA/'
n_experts = 3

weights = []
for i in range(n_experts):
    weights.append(list(np.load(f'{dir}expert_masks/weights{i+1}_percentile.npy')))

files_modif_json = json.load(open('files_modif.json'))
files_modif = files_modif_json[f'{n_experts}_experts']


In [ ]:
max_soft = []
max_smooth = []
max_sc = []


for i,file in enumerate(files_modif):
    print(f'{i}: {file}')
    experts = []
    for exp in range(n_experts):
        experts.append(imread(f'{dir}expert_masks/{exp+1}/{file}'))

    original = imread(f'{dir}train_masks/{file}')



    _,_,soft_final = calculate_soft(experts)
    _,_,smooth_final = calculate_smooth(experts)
    _,_,sc_final = calculate_sc(experts,weights)



    max_soft.append(calculate_max_threshold(original,soft_final))
    max_smooth.append(calculate_max_threshold(original,smooth_final))
    max_sc.append(calculate_max_threshold(original,sc_final))

In [14]:
all_ious_sc = max_sc[0].copy()
all_ious_soft = max_soft[0].copy()
all_ious_smooth = max_smooth[0].copy()


for i in range(1,len(max_sc)):
    for key in max_sc[i].keys():
        all_ious_sc[key] += max_sc[i][key]
        all_ious_soft[key] += max_soft[i][key]
        all_ious_smooth[key] += max_smooth[i][key]

all_ious_sc = {k: v / len(max_sc) for k, v in all_ious_sc.items()}
all_ious_soft = {k: v / len(max_soft) for k, v in all_ious_soft.items()}
all_ious_smooth = {k: v / len(max_smooth) for k, v in all_ious_smooth.items()}

In [ ]:
plt.plot(list(all_ious_sc.keys()), list(all_ious_sc.values()), color='r', label='SC')
plt.plot(list(all_ious_sc.keys()), list(all_ious_soft.values()), color='g', label='Soft')
plt.plot(list(all_ious_sc.keys()), list(all_ious_smooth.values()), color='b', label='Smooth')

# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Threshold (Prob)")
plt.ylabel("Max(FNR,FPR)")
plt.legend()
plt.savefig(f'results_figures/comparison_{n_experts}experts.png')
plt.close("all")